## West Nile Virus Train + Spray EDA

In [1]:
import pandas as pd
import requests
import time
import random
import string
import scipy.stats as stats
import numpy as np
import math 
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
from sklearn.metrics import plot_roc_curve

from ipywidgets import *
from IPython.display import display

%matplotlib inline


In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
weather = pd.read_csv('./data/weather.csv')
spray = pd.read_csv('./data/spray.csv')

In [3]:
# Removinfg first observation date of spray, spray area is very far out from clusters
spray = spray.loc[spray['Date'] != '2011-08-29']

In [13]:
mapdata = np.loadtxt("./data/mapdata_copyright_openstreetmap_contributors.txt")
origin = [41.6, -88.0]              # lat/long of origin (lower left corner)
upperRight = [42.1, -87.5]          # lat/long of upper right corner


def plot(date_obvs, date_spray):
    lats_wnv = np.array(train.loc[(train['Date'] == date_obvs) & (train['WnvPresent'] == 1)]['Latitude'])
    longs_wnv = np.array(train.loc[(train['Date'] == date_obvs) & (train['WnvPresent'] == 1)]['Longitude'])
    
    lats = np.array(train.loc[(train['Date'] == date_obvs) & (train['WnvPresent'] == 0)]['Latitude'])
    longs = np.array(train.loc[(train['Date'] == date_obvs) & (train['WnvPresent'] == 0)]['Longitude'])
                     
    lats_spray = np.array(spray.loc[spray['Date'] == date_spray]['Latitude'])
    longs_spray = np.array(spray.loc[spray['Date'] == date_spray]['Longitude'])
    
    plt.imshow(mapdata, cmap=plt.get_cmap('gray'), extent=[origin[1], upperRight[1], origin[0], upperRight[0]])
    
    plt.scatter(x=longs_wnv, y=lats_wnv, c='r', alpha =0.5, s=10) #Traps with wnv are red
    plt.scatter(x=longs, y=lats, c='b', alpha =0.5, s=10)         #Traps with no wnv are blue
    plt.scatter(x=longs_spray, y=lats_spray, c='y',alpha =0.1, s=10)  #Spray area in yellow
    plt.scatter(x= -87.933, y = 41.995, c='m', s =30)                # Station 1 (purple)
    plt.scatter(x= -87.752, y = 41.786, c='c', s =30)                # Station 2 (blue-ish)
    

In [14]:
interact(plot,
         date_obvs = train['Date'].unique(),
         date_spray = spray['Date'].unique(),
        )

interactive(children=(Dropdown(description='date_obvs', options=('2007-05-29', '2007-06-05', '2007-06-26', '20…

<function __main__.plot(date_obvs, date_spray)>

In [8]:
weather

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2,2014-10-29,49,40,45,M,34,42,20,0,...,,M,M,M,0.00,29.42,30.07,8.5,29,9.0
2940,1,2014-10-30,51,32,42,-4,34,40,23,0,...,,0,M,0.0,0.00,29.34,30.09,5.1,24,5.5
2941,2,2014-10-30,53,37,45,M,35,42,20,0,...,RA,M,M,M,T,29.41,30.10,5.9,23,6.5
2942,1,2014-10-31,47,33,40,-6,25,33,25,0,...,RA SN,0,M,0.1,0.03,29.49,30.20,22.6,34,22.9


In [10]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0
